Market Basket Analysis

Association Analysis

Get our pandas and MLxtend code imported and read the data:

In [1]:
#imports
import pandas as pd #Python data analysis library
import numpy as np #Python scientific computing
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

Citation - “The Instacart Online Grocery Shopping Dataset 2017”, Accessed from https://www.instacart.com/datasets/grocery-shopping-2017"

In [2]:
#import dataset
trainDf = pd.read_csv("../data/raw/order_products__train.csv")
orderDf = pd.read_csv("../data/raw/orders.csv")
productDf = pd.read_csv("../data/raw/products.csv")

We will merge the prior and train dataset to get the complete order dataset. We will use append() for doing so.

In [3]:
priorDf = pd.read_csv("../data/raw/order_products__prior.csv")
trainDf = trainDf.append(priorDf,ignore_index = True)

For counting each product, we can assign reordered column as 1

In [4]:
trainDf['reordered'] = 1 

In [5]:
productCountDf = trainDf.groupby("product_id",as_index = False)["order_id"].count()

In [6]:
newproductCountDf=productCountDf.merge(productDf, left_on='product_id', right_on='product_id', how='inner')
newDf = newproductCountDf[['product_id','product_name']]
newDf

,product_id,product_name
0,1,Chocolate Sandwich Cookies
1,2,All-Seasons Salt
2,3,Robust Golden Unsweetened Oolong Tea
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...
4,5,Green Chile Anytime Sauce
5,6,Dry Nose Oil
6,7,Pure Coconut Water With Orange
7,8,Cut Russet Potatoes Steam N' Mash
8,9,Light Strawberry Blueberry Yogurt
9,10,Sparkling Orange Juice & Prickly Pear Beverage


In [7]:
#Top 100 most frequently purchased products
topLev = 100
productCountDf = productCountDf.sort_values("order_id",ascending = False)
topProdFrame = productCountDf.iloc[0:topLev,:]
productId= topProdFrame.loc[:,["product_id"]]


Now we will filter the orders and get orders containting the the most frequently purchased products

In [8]:
df = trainDf[0:0]
for i in range(0,99):
    pId = productId.iloc[i]['product_id'] 
    stDf = trainDf[trainDf.product_id == pId ]
    df = df.append(stDf,ignore_index = False)

In [9]:
df.head()


,order_id,product_id,add_to_cart_order,reordered
115,226,24852,2,1
156,473,24852,2,1
196,878,24852,2,1
272,1042,24852,1,1
297,1139,24852,1,1


Now we need to consolidate the items into 1 transaction per row with each product 1 hot encoded. Each row will represent an order and each column will represent product_id. If the cell value is '1' say (i,j) then ith order contains jth product.

In [10]:
df=df.merge(newDf, left_on='product_id', right_on='product_id', how='inner')
df=df[['order_id','add_to_cart_order','reordered','product_name']]
df

,order_id,add_to_cart_order,reordered,product_name
0,226,2,1,Banana
1,473,2,1,Banana
2,878,2,1,Banana
3,1042,1,1,Banana
4,1139,1,1,Banana
5,1145,1,1,Banana
6,1468,1,1,Banana
7,1572,3,1,Banana
8,1591,19,1,Banana
9,2068,1,1,Banana


In [11]:
basket = df.groupby(['order_id', 'product_name'])['reordered'].sum().unstack().reset_index().fillna(0).set_index('order_id')

In [12]:
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1 

In [13]:
basket_sets = basket.applymap(encode_units)

In [14]:
basket_sets.head()

product_name,100% Raw Coconut Water,100% Whole Wheat Bread,2% Reduced Fat Milk,Apple Honeycrisp Organic,Asparagus,Bag of Organic Bananas,Banana,Bartlett Pears,Blueberries,Boneless Skinless Chicken Breasts,...,Sparkling Natural Mineral Water,Sparkling Water Grapefruit,Spring Water,Strawberries,Uncured Genoa Salami,Unsalted Butter,Unsweetened Almondmilk,Unsweetened Original Almond Breeze Almond Milk,Whole Milk,Yellow Onions
order_id,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
5,0,0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
basket_sets.size

241667217

Now that the data is structured properly, we can generate frequent item sets that have a support of at least 1%

In [16]:
# Build up the frequent items
frequent_itemsets = apriori(basket_sets, min_support=0.01, use_colnames=True)

In [17]:
frequent_itemsets

,support,itemsets
0,0.016088,[100% Raw Coconut Water]
1,0.025855,[100% Whole Wheat Bread]
2,0.015825,[2% Reduced Fat Milk]
3,0.035751,[Apple Honeycrisp Organic]
4,0.029147,[Asparagus]
5,0.161785,[Bag of Organic Bananas]
6,0.201259,[Banana]
7,0.015002,[Bartlett Pears]
8,0.023870,[Blueberries]
9,0.021453,[Boneless Skinless Chicken Breasts]


The final step is to generate the rules with their corresponding support, confidence and lift:

In [18]:
# Create the rules
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
rules=rules[['antecedants','consequents','antecedent support','consequent support','support','confidence','lift']]

We can also see several where the confidence is high as well.or instance, we can see that there are quite a few rules with a high lift value which means that it occurs more frequently than would be expected given the number of transaction and product combinations.
We can filter the dataframe using standard pandas code. In this case, look for a large lift (2) and high confidence (.1):

In [19]:
rules=rules[ (rules['lift'] >= 2) &
       (rules['confidence'] >= 0.1) ]
rules

,antecedants,consequents,antecedent support,consequent support,support,confidence,lift
6,(Limes),(Large Lemon),0.060080,0.065869,0.011879,0.197723,3.001750
7,(Large Lemon),(Limes),0.065869,0.060080,0.011879,0.180345,3.001750
12,(Organic Blueberries),(Organic Strawberries),0.043024,0.112891,0.010252,0.238274,2.110653
16,(Organic Raspberries),(Organic Hass Avocado),0.058418,0.090483,0.010984,0.188018,2.077938
17,(Organic Hass Avocado),(Organic Raspberries),0.090483,0.058418,0.010984,0.121389,2.077938
22,(Organic Raspberries),(Organic Strawberries),0.058418,0.112891,0.014556,0.249174,2.207206
23,(Organic Strawberries),(Organic Raspberries),0.112891,0.058418,0.014556,0.128940,2.207206
54,(Organic Avocado),(Large Lemon),0.075468,0.065869,0.010555,0.139862,2.123337
55,(Large Lemon),(Organic Avocado),0.065869,0.075468,0.010555,0.160244,2.123337
